In [1]:
import pandas as pd
import numpy as np
import re
import difflib
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, naive_bayes, svm
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import logging
from keras.models import Sequential
from keras import layers
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Full+data.csv')
data.head(1)

,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# reduce to non nan columns
cols = data.columns[:14]
data = data.loc[:,cols]
data.shape

(42373, 14)

In [4]:
# import extra data
extra_data = pd.read_csv('extra_data.csv')
extra_data.head(1)


,product_id,mpn,brand,brand_category,name,labels,updated_at,bc_product_id,saleable,details,created_at,brand_canonical_url,notes,description
0,01E5ZXP5H0BTEZT9QD2HRZJ47A,5529544,A.L.C.,Unknown,Lennox High Waist Cotton & Linen Pants,[],2020-04-17 15:44:57.785000+00:00,5021,2020-04-17 15:44:57.785000+00:00,"True to size. High rise.\n31"" inseam; 14"" leg ...",2020-04-15 21:59:56.695000+00:00,https://shop.nordstrom.com/s/a-l-c-lennox-high...,NaN,High-rise trousers tailored from a cool Italia...


In [5]:
# columns to use from extra_data
cols = ['product_id', 'brand', 'name', 'description', 'brand_category','details']

In [6]:
extra_data = extra_data.loc[:,cols]
data = data.loc[:,cols]

In [7]:
# concatenate data and extra data
full_data = pd.concat([data,extra_data])
len(full_data) == len(data) + len(extra_data)

True

In [8]:
full_data.fillna('UNKNOWNTOKEN',inplace=True)

In [9]:
# we will join brand and brand_category together to brand_info for use in predictions

#full_data['brand_info'] = full_data['brand']+full_data['brand_category']+ " " +full_data['brand_canonical_url']

In [10]:
full_data.isnull().sum()

product_id        0
brand             0
name              0
description       0
brand_category    0
details           0
dtype: int64

In [11]:
full_data = full_data.reset_index(drop = True)

In [12]:
tags_style_all = pd.read_csv('color.csv',index_col = 0)
data_color = pd.merge(full_data,tags_style_all,on='product_id', how = 'inner')
#data_label.to_csv('data_label.csv')

In [13]:
data_color.shape

(39956, 28)

In [14]:
# function to remove stopwords
from gensim.parsing.preprocessing import STOPWORDS
stop = set(STOPWORDS)
def remove_stopwords(data_col):
    new_list = []
    a = data_col
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            if word not in stop:
                res_words.append(word)
            sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [15]:
# remove stopwords from each of the 4 columns
data_color['description'] = remove_stopwords(data_color['description'])
data_color['details'] = remove_stopwords(data_color['details'])
data_color['name'] = remove_stopwords(data_color['name'])
data_color['brand'] = remove_stopwords(data_color['brand'])
data_color['brand_category'] = remove_stopwords(data_color['brand_category'])

In [16]:
# function to do regex cleaning
def reg_clean(data,col):
    new_list = []
    for i in range(0,len(data)):
        #special characters 
        a = re.sub(r'[^ a-zA-Z0-9]',' ',data.loc[i,col])
        #new lines
        a = re.sub(r'\n',' ', a)
        #remove multiple spaces by a single space
        a = re.sub(r'\s+',' ',a)
        #timestamp
        a = re.sub(r'\b[0-9]{1,}am|[0-9]{1,}pm|[0-9]{4,}|[0-9]ish|1st|2nd|3rd|[0-9]{1,2}th|31st|[0-9]{1,}min(?:utes)?s?|[0-9]{1,}h(?:ou)?rs?|[0-9]{3,}\b','timestamp',a)
        a = re.sub(r'\b[0-9]{1,}timestamp\b','timestamp',a)
        #any numbers as digit
        a = re.sub(r'\b\d+\b','digit',a)
        #number followed by a variable
        a = re.sub(r'\b\d{1,}[a-z]{0,}[0-9]{0,}','varchar',a)
        #html codes
        a = re.sub(r'<.+?>','html',a)
        a = re.sub(r'https|www|com','html',a)
        # additional stopwords
        a = re.sub(r'\ba\b', '', a, flags = re.IGNORECASE)
        a = re.sub(r'\bthis|the\b', '',a, flags = re.IGNORECASE)
        new_list.append(a)
    return new_list

In [17]:
# perform regex cleaning on each column
data_color['description'] = reg_clean(data_color,'description')
data_color['details'] = reg_clean(data_color,'details')
data_color['brand'] = reg_clean(data_color,'brand')
data_color['name'] = reg_clean(data_color, 'name')
data_color['brand_category'] = reg_clean(data_color,'brand_category')

In [18]:
# function to lemmatize columns
def lemmatize_sentence(data_col):
    new_list = []
    a = data_col 
    for i in range(0,len(a)):
        words = word_tokenize(a[i])
        res_words = []
        for word in words:
            res_words.append(lemmatizer.lemmatize(word).strip(string.punctuation))
        sentence = " ".join(res_words)
        new_list.append(sentence)
    return new_list

In [19]:
# lemmatize each column
data_color['description'] = lemmatize_sentence(data_color['description'])
data_color['details'] = lemmatize_sentence(data_color['details'])
data_color['brand'] = lemmatize_sentence(data_color['brand'])
data_color['brand_category'] = lemmatize_sentence(data_color['brand_category'])
data_color['name'] = lemmatize_sentence(data_color['name'])

In [20]:
# function to start collocation
stopwords_coll = set(stopwords.words('english') + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-","timestamp","varchar","html","digit"])
filter_stops = lambda w: len(w) < 3 or w in stopwords_coll
def collocation_list(data_col):
    new_list = []
    for i in range(0,len(data_col)):
        words = word_tokenize(data_col[i])
        res_words = []
        for word in words:
            if word not in stopwords_coll:
                res_words.append(word)
        new_list.append(res_words)
    return(new_list)

In [21]:
cols = ['brand', 'brand_category', 'description', 'name','details']
n_list = []
for col in cols:
    n_list.append(collocation_list(full_data[col]))

In [22]:
# we find that dry clean, machine wash, 'fits true to size', 'made in italy' 
# should be collated together

col_list = [['Dry', 'clean'], ['Machine', 'wash'], ['true', 'size'],['fit', 'true_size'], ['Fits', 'true_size'],  
            ['Made', 'Italy']]
data_col = data_color['details']
new_list = []
clean_list = ['Dry','wash','Italy', 'size']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['details'] = new_list

In [23]:
# we find that wide leg, ankle boot, high waist, high rise 
# should be collated together

col_list = [['wide','leg'], ['ankle', 'boot'], ['High', 'Waist'], ['Wide', 'Leg'], ['High', 'Rise'], 
            ['straight', 'leg'], ['shoulder', 'bag']]
data_col = data_color['name']
new_list = []
clean_list = ['leg', 'boot', 'Waist', 'Leg', 'Rise','bag']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['name'] = new_list

In [24]:
col_list = [['Dry','clean'], ['Machine', 'wash'], ['Made', 'Italy'], ['wide', 'leg'], ['best', 'selling'], 
            ['high', 'waisted'], ['pencil', 'skirt'], ['High', 'Rise'], ['high','rise'], ['Hand','wash']]
data_col = data_color['description']
new_list = []
clean_list = ['clean', 'wash', 'Italy', 'leg', 'selling','waisted', 'skirt','Rise', 'rise']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['description'] = new_list

In [25]:
# we find that  should be collocated together

col_list = [['New', 'York'], ['Jimmy','Choo'], ['Banana','Republic'], ['Victoria','Beckham']]
data_col = data_color['brand']
new_list = []
clean_list = ['York', 'Choo','Republic', 'Beckham']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand'] = new_list

In [26]:
col_list = [['Mid', 'Heel'], ['Shoulder', 'Bags']]
data_col = data_color['brand_category']
new_list = []
clean_list = ['Heel', 'Bags']
for i in range(0,len(data_col)):
    words = word_tokenize(data_col[i])
    len_words = len(words)-1
    for i in range(0,len_words):
        bi_word = []
        j = i+1
        bi_word.append(words[i])
        bi_word.append(words[j])
        if(bi_word in col_list):
            sentence = "_".join(bi_word)
            words[i] = sentence
    for word in words:
        if word in clean_list:
            words.remove(word)
    res_word = []
    for word in words:
        res_word.append(word)
    sentence = " ".join(res_word)
    new_list.append(sentence)
data_color['brand_category'] = new_list

In [27]:
data_color.isna().sum()

product_id           0
brand                0
name                 0
description          0
brand_category       0
details              0
product_color_id     0
attribute_name       0
attribute_value      0
is_black            14
is_beige            14
is_burgundy         14
is_white            14
is_gray             14
is_gold             14
is_blue             14
is_neutral          14
is_pink             14
is_orange           14
is_navy             14
is_brown            14
is_red              14
is_yellow           14
is_multi            14
is_green            14
is_silver           14
is_teal             14
is_purple           14
dtype: int64

In [28]:
data_color.fillna(0, inplace=True)

In [29]:
data_color.isna().sum()

product_id          0
brand               0
name                0
description         0
brand_category      0
details             0
product_color_id    0
attribute_name      0
attribute_value     0
is_black            0
is_beige            0
is_burgundy         0
is_white            0
is_gray             0
is_gold             0
is_blue             0
is_neutral          0
is_pink             0
is_orange           0
is_navy             0
is_brown            0
is_red              0
is_yellow           0
is_multi            0
is_green            0
is_silver           0
is_teal             0
is_purple           0
dtype: int64

In [30]:
#impute 0's to na's 

In [31]:
# initialize the tf-idf vectorizer
vectorizer = TfidfVectorizer(token_pattern=r'\b[a-zA-Z]{3,}\b',stop_words="english",binary = True,min_df = 0.005,max_df = 0.7,max_features =300)

In [32]:
# vectorizing our test/full data
columns = ['description','details','brand', 'brand_category', 'name']
model_data=pd.DataFrame()
for j in columns:
    corpus = []
    for i in range(0,len(data_color)):
        corpus.append(data_color.loc[i,j])
    vect = vectorizer.fit_transform(corpus)
    terms = vectorizer.get_feature_names()
    c=pd.DataFrame(vect.toarray().transpose(), index=terms)
    model_data=pd.concat([model_data,c.T],axis = 1)

In [33]:
model_data.shape

(39956, 890)

In [34]:
data_color.shape

(39956, 28)

In [35]:
data_color.columns[9:]

Index(['is_black', 'is_beige', 'is_burgundy', 'is_white', 'is_gray', 'is_gold',
       'is_blue', 'is_neutral', 'is_pink', 'is_orange', 'is_navy', 'is_brown',
       'is_red', 'is_yellow', 'is_multi', 'is_green', 'is_silver', 'is_teal',
       'is_purple'],
      dtype='object')

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
logreg=LogisticRegression(n_jobs=1, C=1e5)  

In [38]:
cols = data_color.columns[9:]
for col in cols:
    x=model_data
    y=data_color[col].values
    logreg.fit(x,y)
    # predict the labels on validation dataset
    predictions_log = logreg.predict(x)
    data_color[col+'_predicted'] = predictions_log

In [39]:
data_color.shape

(39956, 47)

In [40]:
data_color.to_csv('data_color_predicted.csv')

In [41]:
#training_data = pd.read_csv('subset_color.csv')